**Flowsheet Containing a Single Mixer**

**Problem Statement**: Consider the process of mixing benzene and toluene streams to form a mixture with the following inlet specificatons:

**Stream 1:**
Component: Benzene 
Flow Rate = 0.6 kmol/hr

**Stream 2**
Component: Toluene 
Flow Rate = 0.4 kmol/hr

Pressure = 101325 Pa
Temperature = 353 K

We will study the simulation of the mixer by fixing the above specifications 

IDAES documentation reference for mixer model: https://idaes-pse.readthedocs.io/en/stable/models/mixer.html

**Setting up the problem in IDAES**

In [1]:
# Import pyomo package 
from pyomo.environ import ConcreteModel, SolverFactory, Constraint, value

# Import the main FlowsheetBlock from IDAES. The flowsheet block will contain the unit model
from idaes.core import FlowsheetBlock

#Create the ConcreteModel and the FlowsheetBlock, and attach the flowsheet block to it.
m = ConcreteModel()
m.fs = FlowsheetBlock(default={"dynamic": False})
# Steady State Model

#import the BTX property package to create a properties block for the flowsheet
from idaes.generic_models.properties.activity_coeff_models import BTX_activity_coeff_VLE

# Add properties as parameters to flowsheet directly, and attach the properties block to the flowsheet
m.fs.properties = BTX_activity_coeff_VLE.BTXParameterBlock(default={"valid_phase":
                                                     'Liq',
                                                     "activity_coeff_model":
                                                     "Ideal"})
#Import a mixer unit
from idaes.generic_models.unit_models.mixer import Mixer
#Create an instance of the mixer unit, attaching it to the flowsheet
#specify that the property package to be used with the mixer is the one we created earlier.
m.fs.mixer = Mixer(default={"property_package": m.fs.properties,"num_inlets":2})

#import the degrees_of_freedom function from the idaes.core.util.model_statistics package
from idaes.core.util.model_statistics import degrees_of_freedom
# DOF = Number of Model Variables - Number of Model Constraints

# Call the degrees_of_freedom function, get initial DOF
DOF_initial = degrees_of_freedom(m)

#Fix the stream inlet conditions
m.fs.mixer.inlet_1.flow_mol.fix(0.6*1000/3600)
m.fs.mixer.inlet_1.mole_frac_comp[0, "benzene"].fix(1)
m.fs.mixer.inlet_1.mole_frac_comp[0, "toluene"].fix(0)
m.fs.mixer.inlet_1.pressure.fix(101325)
m.fs.mixer.inlet_1.temperature.fix(353)
m.fs.mixer.inlet_2.flow_mol.fix(0.4*1000/3600)
m.fs.mixer.inlet_2.mole_frac_comp[0, "benzene"].fix(0)
m.fs.mixer.inlet_2.mole_frac_comp[0, "toluene"].fix(1)
m.fs.mixer.inlet_2.pressure.fix(101325)
m.fs.mixer.inlet_2.temperature.fix(353)

# Call the degrees_of_freedom function, get final DOF
DOF_final = degrees_of_freedom(m)

**Obtain Simulation Results**

In [2]:
#Initialize the flowsheet 
m.fs.mixer.initialize()
#Solve the simulation using ipopt
#Note: If the degrees of freedom = 0, we have a square problem
opt = SolverFactory('ipopt')
opt.solve(m)

2020-03-06 13:45:59 - INFO - idaes.init.fs.mixer.inlet_1_state - Initialization Step 1 skipped.
2020-03-06 13:46:00 - DEBUG - idaes.solve.fs.mixer.inlet_1_state - Ipopt 3.12.13:
2020-03-06 13:46:00 - DEBUG - idaes.solve.fs.mixer.inlet_1_state - ******************************************************************************
2020-03-06 13:46:00 - DEBUG - idaes.solve.fs.mixer.inlet_1_state - This program contains Ipopt, a library for large-scale nonlinear optimization.
2020-03-06 13:46:00 - DEBUG - idaes.solve.fs.mixer.inlet_1_state - Ipopt is released as open source code under the Eclipse Public License (EPL).
2020-03-06 13:46:00 - DEBUG - idaes.solve.fs.mixer.inlet_1_state - For more information visit http://projects.coin-or.org/Ipopt
2020-03-06 13:46:00 - DEBUG - idaes.solve.fs.mixer.inlet_1_state - This version of Ipopt was compiled from source code available at
2020-03-06 13:46:00 - DEBUG - idaes.solve.fs.mixer.inlet_1_state - https://github.com/IDAES/Ipopt as part of the Institute fo

2020-03-06 13:46:00 - DEBUG - idaes.solve.fs.mixer.inlet_1_state - Framework) Copyright (c) 2018-2019. See https://github.com/IDAES/idaes-pse.
2020-03-06 13:46:00 - DEBUG - idaes.solve.fs.mixer.inlet_1_state - This version of Ipopt was compiled using HSL, a collection of Fortran codes
2020-03-06 13:46:00 - DEBUG - idaes.solve.fs.mixer.inlet_1_state - for large-scale scientific computation.  All technical papers, sales and
2020-03-06 13:46:00 - DEBUG - idaes.solve.fs.mixer.inlet_1_state - publicity material resulting from use of the HSL codes within IPOPT must
2020-03-06 13:46:00 - DEBUG - idaes.solve.fs.mixer.inlet_1_state - contain the following acknowledgement:
2020-03-06 13:46:00 - DEBUG - idaes.solve.fs.mixer.inlet_1_state - HSL, a collection of Fortran codes for large-scale scientific
2020-03-06 13:46:00 - DEBUG - idaes.solve.fs.mixer.inlet_1_state - computation. See http://www.hsl.rl.ac.uk.
2020-03-06 13:46:00 - DEBUG - idaes.solve.fs.mixer.inlet_1_state - ***********************

2020-03-06 13:46:00 - DEBUG - idaes.solve.fs.mixer.inlet_1_state - Number of nonzeros in Lagrangian Hessian.............:        2
2020-03-06 13:46:00 - DEBUG - idaes.solve.fs.mixer.inlet_1_state - Total number of variables............................:        5
2020-03-06 13:46:00 - DEBUG - idaes.solve.fs.mixer.inlet_1_state - variables with only lower bounds:        0
2020-03-06 13:46:00 - DEBUG - idaes.solve.fs.mixer.inlet_1_state - variables with lower and upper bounds:        2
2020-03-06 13:46:00 - DEBUG - idaes.solve.fs.mixer.inlet_1_state - variables with only upper bounds:        0
2020-03-06 13:46:00 - DEBUG - idaes.solve.fs.mixer.inlet_1_state - Total number of equality constraints.................:        5
2020-03-06 13:46:00 - DEBUG - idaes.solve.fs.mixer.inlet_1_state - Total number of inequality constraints...............:        0
2020-03-06 13:46:00 - DEBUG - idaes.solve.fs.mixer.inlet_1_state - inequality constraints with only lower bounds:        0
2020-03-06 13:46:0

2020-03-06 13:46:00 - DEBUG - idaes.solve.fs.mixer.inlet_1_state - 1  0.0000000e+00 5.65e+02 1.28e+00  -1.0 5.68e+04    -  1.00e+00 9.90e-01h  1
2020-03-06 13:46:00 - DEBUG - idaes.solve.fs.mixer.inlet_1_state - 2  0.0000000e+00 5.59e+00 9.90e+00  -1.0 5.68e+02    -  1.00e+00 9.90e-01h  1
2020-03-06 13:46:00 - DEBUG - idaes.solve.fs.mixer.inlet_1_state - 3  0.0000000e+00 1.22e-09 2.38e-01  -1.7 5.62e+00    -  1.00e+00 1.00e+00h  1
2020-03-06 13:46:00 - DEBUG - idaes.solve.fs.mixer.inlet_1_state - Number of Iterations....: 3
2020-03-06 13:46:00 - DEBUG - idaes.solve.fs.mixer.inlet_1_state - (scaled)                 (unscaled)
2020-03-06 13:46:00 - DEBUG - idaes.solve.fs.mixer.inlet_1_state - Objective...............:   0.0000000000000000e+00    0.0000000000000000e+00
2020-03-06 13:46:00 - DEBUG - idaes.solve.fs.mixer.inlet_1_state - Dual infeasibility......:   0.0000000000000000e+00    0.0000000000000000e+00
2020-03-06 13:46:00 - DEBUG - idaes.solve.fs.mixer.inlet_1_state - Constraint v

2020-03-06 13:46:00 - DEBUG - idaes.solve.fs.mixer.inlet_2_state - Complementarity.........:   0.0000000000000000e+00    0.0000000000000000e+00
2020-03-06 13:46:00 - DEBUG - idaes.solve.fs.mixer.inlet_2_state - Overall NLP error.......:   0.0000000000000000e+00    0.0000000000000000e+00
2020-03-06 13:46:00 - DEBUG - idaes.solve.fs.mixer.inlet_2_state - Number of objective function evaluations             = 6
2020-03-06 13:46:00 - DEBUG - idaes.solve.fs.mixer.inlet_2_state - Number of objective gradient evaluations             = 6
2020-03-06 13:46:00 - DEBUG - idaes.solve.fs.mixer.inlet_2_state - Number of equality constraint evaluations            = 6
2020-03-06 13:46:00 - DEBUG - idaes.solve.fs.mixer.inlet_2_state - Number of inequality constraint evaluations          = 0
2020-03-06 13:46:00 - DEBUG - idaes.solve.fs.mixer.inlet_2_state - Number of equality constraint Jacobian evaluations   = 6
2020-03-06 13:46:00 - DEBUG - idaes.solve.fs.mixer.inlet_2_state - Number of inequality cons

...

2020-03-06 13:46:01 - DEBUG - idaes.solve.fs.mixer.mixed_state - Total number of variables............................:        5
2020-03-06 13:46:01 - DEBUG - idaes.solve.fs.mixer.mixed_state - variables with only lower bounds:        0
2020-03-06 13:46:01 - DEBUG - idaes.solve.fs.mixer.mixed_state - variables with lower and upper bounds:        2
2020-03-06 13:46:01 - DEBUG - idaes.solve.fs.mixer.mixed_state - variables with only upper bounds:        0
2020-03-06 13:46:01 - DEBUG - idaes.solve.fs.mixer.mixed_state - Total number of equality constraints.................:        5
2020-03-06 13:46:01 - DEBUG - idaes.solve.fs.mixer.mixed_state - Total number of inequality constraints...............:        0
2020-03-06 13:46:01 - DEBUG - idaes.solve.fs.mixer.mixed_state - inequality constraints with only lower bounds:        0
2020-03-06 13:46:01 - DEBUG - idaes.solve.fs.mixer.mixed_state - inequality constraints with lower and upper bounds:        0
2020-03-06 13:46:01 - DEBUG - idaes.s

2020-03-06 13:46:01 - DEBUG - idaes.solve.fs.mixer.mixed_state - Constraint violation....:   3.7252902984619143e-10    3.7252902984619141e-09
2020-03-06 13:46:01 - DEBUG - idaes.solve.fs.mixer.mixed_state - Complementarity.........:   0.0000000000000000e+00    0.0000000000000000e+00
2020-03-06 13:46:01 - DEBUG - idaes.solve.fs.mixer.mixed_state - Overall NLP error.......:   3.7252902984619143e-10    3.7252902984619141e-09
2020-03-06 13:46:01 - DEBUG - idaes.solve.fs.mixer.mixed_state - Number of objective function evaluations             = 1
2020-03-06 13:46:01 - DEBUG - idaes.solve.fs.mixer.mixed_state - Number of objective gradient evaluations             = 1
2020-03-06 13:46:01 - DEBUG - idaes.solve.fs.mixer.mixed_state - Number of equality constraint evaluations            = 1
2020-03-06 13:46:01 - DEBUG - idaes.solve.fs.mixer.mixed_state - Number of inequality constraint evaluations          = 0
2020-03-06 13:46:01 - DEBUG - idaes.solve.fs.mixer.mixed_state - Number of equality co

2020-03-06 13:46:01 - DEBUG - idaes.solve.fs.mixer.mixed_state - This program contains Ipopt, a library for large-scale nonlinear optimization.
2020-03-06 13:46:01 - DEBUG - idaes.solve.fs.mixer.mixed_state - Ipopt is released as open source code under the Eclipse Public License (EPL).
2020-03-06 13:46:01 - DEBUG - idaes.solve.fs.mixer.mixed_state - For more information visit http://projects.coin-or.org/Ipopt
2020-03-06 13:46:01 - DEBUG - idaes.solve.fs.mixer.mixed_state - This version of Ipopt was compiled from source code available at
2020-03-06 13:46:01 - DEBUG - idaes.solve.fs.mixer.mixed_state - https://github.com/IDAES/Ipopt as part of the Institute for the Design of
2020-03-06 13:46:01 - DEBUG - idaes.solve.fs.mixer.mixed_state - Advanced Energy Systems Process Systems Engineering Framework (IDAES PSE
2020-03-06 13:46:01 - DEBUG - idaes.solve.fs.mixer.mixed_state - Framework) Copyright (c) 2018-2019. See https://github.com/IDAES/idaes-pse.
2020-03-06 13:46:01 - DEBUG - idaes.sol

2020-03-06 13:46:01 - DEBUG - idaes.solve.fs.mixer - This is Ipopt version 3.12.13, running with linear solver ma27.
2020-03-06 13:46:01 - DEBUG - idaes.solve.fs.mixer - Number of nonzeros in equality constraint Jacobian...:       68
2020-03-06 13:46:01 - DEBUG - idaes.solve.fs.mixer - Number of nonzeros in inequality constraint Jacobian.:        0
2020-03-06 13:46:01 - DEBUG - idaes.solve.fs.mixer - Number of nonzeros in Lagrangian Hessian.............:       19
2020-03-06 13:46:01 - DEBUG - idaes.solve.fs.mixer - Total number of variables............................:       25
2020-03-06 13:46:01 - DEBUG - idaes.solve.fs.mixer - variables with only lower bounds:        3
2020-03-06 13:46:01 - DEBUG - idaes.solve.fs.mixer - variables with lower and upper bounds:        8
2020-03-06 13:46:01 - DEBUG - idaes.solve.fs.mixer - variables with only upper bounds:        0
2020-03-06 13:46:01 - DEBUG - idaes.solve.fs.mixer - Total number of equality constraints.................:       25
2020-

{'Problem': [{'Lower bound': -inf, 'Upper bound': inf, 'Number of objectives': 1, 'Number of constraints': 25, 'Number of variables': 25, 'Sense': 'unknown'}], 'Solver': [{'Status': 'ok', 'Message': 'Ipopt 3.12.13\\x3a Optimal Solution Found', 'Termination condition': 'optimal', 'Id': 0, 'Error rc': 0, 'Time': 0.03195667266845703}], 'Solution': [OrderedDict([('number of solutions', 0), ('number of solutions displayed', 0)])]}

In [14]:
#Display the outlet stream flowrate (mol/s), pressure (Pa), temperature (K), and enthalpy
m.fs.mixer.outlet.flow_mol.display()
m.fs.mixer.outlet.pressure.display()
m.fs.mixer.outlet.temperature.display()

_outlet_flow_mol_ref : Size=1, Index=fs.time
    Key : Lower : Value              : Upper : Fixed : Stale : Domain
    0.0 :     0 : 0.2777777777777778 :  None : False : False : NonNegativeReals
_outlet_pressure_ref : Size=1, Index=fs.time
    Key : Lower : Value       : Upper : Fixed : Stale : Domain
    0.0 :     0 : 101324.9995 :  None : False : False : NonNegativeReals
_outlet_temperature_ref : Size=1, Index=fs.time
    Key : Lower : Value              : Upper : Fixed : Stale : Domain
    0.0 :     0 : 352.99999999999994 :  None : False : False : NonNegativeReals


In [5]:
#Display a readable report
m.fs.mixer.report()


Unit : fs.mixer                                                            Time: 0.0
------------------------------------------------------------------------------------
    Stream Table
                             inlet_1    inlet_2    Outlet  
    flow_mol                  0.16667    0.11111    0.27778
    mole_frac_comp benzene     1.0000     0.0000    0.60000
    mole_frac_comp toluene     0.0000     1.0000    0.40000
    temperature                353.00     353.00     353.00
    pressure               1.0132e+05 1.0132e+05 1.0132e+05
